In [2]:

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout,Embedding, GRU, BatchNormalization, Input
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from tensorflow.keras.activations import relu, softmax, gelu
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import mean_squared_error, categorical_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras.layers import Concatenate, concatenate
import numpy as np


In [3]:
class upsample_conv(Layer):
    def __init__(self, pool_size = 4, lags = 256, internal_dim = 128, activation = "gelu", dropout_rate = 0.2):
        assert (lags/pool_size).is_integer(), "Lag size should be divisible by pool_size"
        super().__init__()
        self.conv = Conv1D(filters = internal_dim, kernel_size = pool_size, strides = pool_size, use_bias = False)
        self.activation = tf.keras.activations.get(activation)
        self.norm = BatchNormalization()
        self.embed = Embedding(int(lags/pool_size), internal_dim)
        self.list = np.array([i for i in range(int(lags/pool_size))])
        self.dropout = Dropout(dropout_rate)
    @tf.function(jit_compile = True)
    def call(self, inputs, training = None):
        x = self.dropout(inputs, training)
        x = self.conv(inputs)
        x += self.embed(self.list)
        x = self.activation(x)
        x = self.norm(x, training)
        return x